In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

In [4]:
df = pd.read_excel('../data/wachttijden.xlsx')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200433 entries, 0 to 200432
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ID                         200433 non-null  float64       
 1   NAAM_VESTIGING             200433 non-null  object        
 2   KVK_VESTIGING_NR           200429 non-null  object        
 3   TYPE_WACHTTIJD             200433 non-null  object        
 4   CODE_WACHTTIJD             200433 non-null  object        
 5   WACHTTIJD                  188934 non-null  float64       
 6   OMS_WACHTTIJD              200433 non-null  object        
 7   SPECIALISME                200433 non-null  object        
 8   LOCATIECODE                200433 non-null  object        
 9   AGBCODE                    200433 non-null  int64         
 10  STRAAT                     200433 non-null  object        
 11  POSTCODE                   200433 non-null  object  

In [7]:
df_features = df.drop(['ID','NAAM_VESTIGING','KVK_VESTIGING_NR','CODE_WACHTTIJD','LOCATIECODE','AGBCODE','STRAAT','POSTCODE','PLAATS','DATUM_ZORGBEELD','WEEK_DAGEN','JAARMAAND','PEILDATUM','INSUFFICIENT_OBSERVATIONS'], axis=1)

In [8]:
df_features.head(5)

,TYPE_WACHTTIJD,WACHTTIJD,OMS_WACHTTIJD,SPECIALISME,ROAZ_REGIO,NAAM_INSTELLING,TYPE_ZORGINSTELLING
0,Behandeling,90.0,Tonsillectomie en/of adenotomie (kno),KNO (302),Noord-NL,Wilhelmina Ziekenhuis,Ziekenhuis
1,Polikliniekbezoek,21.0,Orthopedie - Heupklachten,Orthopedie (305),Oost,Stichting Rijnstate Ziekenhuis,Ziekenhuis
2,Behandeling,166.0,Initiële staaroperatie (oogheelkunde),Oogheelkunde (301),SpzNet AMC,Stichting Dijklander Ziekenhuis,Ziekenhuis
3,Behandeling,14.0,Triggerfingerrelease (plastische chirurgie),Plastische Chirurgie (304),Brabant,Xpert Clinics Hand- en polszorg B.V.,Kliniek
4,Polikliniekbezoek,28.0,Chirurgie (heelkunde) - Proctologie,Chirurgie (heelkunde) (303),Noord-NL,Ziekenhuis Nij Smellinghe,Ziekenhuis


In [9]:
# Determine how to fill in the missing values.
# We think the missing values should be replaced with the mean of the column.
df_features.groupby('OMS_WACHTTIJD').median('WACHTTIJD')

,WACHTTIJD
OMS_WACHTTIJD,
(Vaat)chirurgische behandeling spataderen (heelkunde),32.0
Ablatie (cardiologie),80.0
Anesthesiologie / Pijnbestrijding,21.0
"Anti-snurkbehandeling (CPAP, operatieve behandeling) (kno)",32.0
Apexresectie(s) (kaakchirurgie),28.0
...,...
Transgender zorg,70.0
Triggerfingerrelease (plastische chirurgie),21.0
Urologie - algemeen,35.0


In [10]:
# Fill in missing values with the median. This seems to be the best way to fill in the missing values.
df_features['WACHTTIJD'] = df_features.groupby('OMS_WACHTTIJD')['WACHTTIJD'].transform(lambda x: x.fillna(x.median()))

In [11]:
# Find outliers in wachttijd.
df_features[df_features['WACHTTIJD'] > 500].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 296 to 196780
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TYPE_WACHTTIJD       155 non-null    object 
 1   WACHTTIJD            155 non-null    float64
 2   OMS_WACHTTIJD        155 non-null    object 
 3   SPECIALISME          155 non-null    object 
 4   ROAZ_REGIO           155 non-null    object 
 5   NAAM_INSTELLING      155 non-null    object 
 6   TYPE_ZORGINSTELLING  155 non-null    object 
dtypes: float64(1), object(6)
memory usage: 9.7+ KB


In [12]:
# Remove outlier observations
# After checking zorgkaart nederland, we found that this can happen for real.
# df_cleaned = df_features[df_features['WACHTTIJD'] < 500]

In [20]:
# df_features['WACHTTIJD'] = df_features.groupby('OMS_WACHTTIJD')['WACHTTIJD'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df_features.dropna(subset=['WACHTTIJD'], inplace=True)

In [21]:
# Fix missing value for Heartlife, it turns out to be a Kliniek!
df_features['TYPE_ZORGINSTELLING'].fillna('Kliniek', inplace=True)

In [23]:
profile = ProfileReport(df_features, title="Feature profiling report")
profile.to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
df_cleaned.to_csv('../data/preprocessed.csv', header=True, index=False)